In [1]:
import datetime
import numpy as np
import pandas as pd

from numpy import genfromtxt



In [80]:
# landcover data can be accessed on kaggle website
# https://www.kaggle.com/crawford/deepsat-sat6

x_train = genfromtxt('./data/X_train_sat6.csv', delimiter=',', max_rows=10e4)
y_train = genfromtxt('./data/y_train_sat6.csv', delimiter=',', max_rows=10e4)
print(x_train.shape)
print(y_train.shape)

(100000, 3136)
(100000, 6)


In [81]:
x_train = x_train.reshape(x_train.shape[0],28,28,4).astype(np.float32)
y_train = y_train.astype(np.float32)
print(x_train.shape)
print(y_train.shape)

(100000, 28, 28, 4)
(100000, 6)


In [83]:
x_val = genfromtxt('./data/X_test_sat6.csv', delimiter=',', max_rows=10e4)
y_val = genfromtxt('./landcover_data/y_test_sat6.csv', delimiter=',', max_rows=10e4)
print(x_val.shape)
print(y_val.shape)

(81000, 3136)
(81000, 6)


In [84]:
x_val = x_val.reshape(x_val.shape[0],28,28,4).astype(np.float32)
y_val = y_val.astype(np.float32)
print(x_val.shape)
print(y_val.shape)

(81000, 28, 28, 4)
(81000, 6)


In [ ]:
# building,1,0,0,0,0,0
# barren_land,0,1,0,0,0,0
# trees,0,0,1,0,0,0
# grassland,0,0,0,1,0,0
# road,0,0,0,0,1,0
# water,0,0,0,0,0,1

In [85]:
with open('./landcover_data/x_train.npy', 'wb') as f:
    np.save(f, x_train)


In [86]:
with open('./landcover_data/y_train.npy', 'wb') as f:
    np.save(f, y_train)

In [87]:
with open('./landcover_data/x_val.npy', 'wb') as f:
    np.save(f, x_val)

In [88]:
with open('./landcover_data/y_val.npy', 'wb') as f:
    np.save(f, y_val)

In [91]:
# print label count

count = np.sum(y_train, axis=0)
count

array([ 4676., 22516., 17588., 15681.,  2507., 37032.], dtype=float32)

In [92]:
import tensorflow as tf
from tensorflow import keras as keras
from tensorflow.keras.optimizers import Adam

from keras import regularizers
from keras.callbacks import ModelCheckpoint, History, ReduceLROnPlateau, TensorBoard, EarlyStopping
from keras.layers import BatchNormalization, Input, Reshape, Dense, Conv2D, Dropout
from keras.models import Model, load_model
from tensorflow.keras.metrics import CategoricalAccuracy, CategoricalCrossentropy, SensitivityAtSpecificity

from timeit import default_timer as timer

In [93]:
LR = 5e-4
MODEL_CHECKPOINT = './snapshots/'
LOG_DIR = './logs/'


In [94]:
def build_cnn():
    l = Input(shape=(28, 28, 4,))
    inputs = l
    
    l = Conv2D(
        filters=32,kernel_size=[5,5],
        padding='same',activation='relu', name='conv1')(l)
    l = BatchNormalization()(l)
    l = Dropout(0.5)(l)

    
    l = Conv2D(
        filters=64,kernel_size=[5,5],
        padding='same',activation='relu', name='conv2')(l)
    l = BatchNormalization()(l)
    
    
    l = Conv2D(
        filters=32,kernel_size=[5,5],
        padding='same',activation='relu', name='conv3')(l)
    l = BatchNormalization()(l)
    
    
    l = Conv2D(
        filters=8,kernel_size=[5,5],
        padding='same',activation='relu', name='conv4')(l)
    l = BatchNormalization()(l)
    l = Dropout(0.5)(l)
    
    l = Reshape((8*28*28,), name='flatten')(l)
    l = Dense(32, activation="relu", name='fc')(l)
    l = Dense(6, activation="softmax", name='main_output')(l)


    m_cnn = Model(inputs=inputs, outputs=l)
    print(m_cnn.summary())
    return m_cnn


In [95]:
def model_train(X_train, y_train, X_val, y_val, model, key):
    print(model.summary())

    file_path = MODEL_CHECKPOINT + key + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    history = History()
    log_dir = LOG_DIR + key + '/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensor_board = TensorBoard(log_dir, histogram_freq=5,
                                       write_grads=False, write_graph=False)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, min_lr=1e-5)
    checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min', period=1)
    early_stop = EarlyStopping(
        monitor="val_loss", min_delta=0, patience=3, verbose=0, mode="auto", baseline=None, restore_best_weights=True
    )
    callbacks_list = [history, tensor_board, reduce_lr, early_stop]

    adam_wn = Adam(learning_rate=LR)
    model.compile(
        loss="categorical_crossentropy", optimizer=adam_wn,
        metrics=[CategoricalAccuracy(), SensitivityAtSpecificity(specificity=0.99, class_id=4)]
    )
    model.fit(x=X_train, y=y_train, validation_data=(X_val, y_val), batch_size=128,
              epochs=40, verbose=1, shuffle=False,
              callbacks=callbacks_list)
    model.save(file_path)

    return (model, history)

In [96]:
my_model = build_cnn()


Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        [(None, 28, 28, 4)]       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 28, 28, 32)        3232      
_________________________________________________________________
batch_normalization_72 (Batc (None, 28, 28, 32)        128       
_________________________________________________________________
dropout_36 (Dropout)         (None, 28, 28, 32)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 28, 28, 64)        51264     
_________________________________________________________________
batch_normalization_73 (Batc (None, 28, 28, 64)        256       
_________________________________________________________________
conv3 (Conv2D)               (None, 28, 28, 32)        512

In [97]:
model_train(x_train, y_train, x_val, y_val, my_model, key='toy_model2')

Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        [(None, 28, 28, 4)]       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 28, 28, 32)        3232      
_________________________________________________________________
batch_normalization_72 (Batc (None, 28, 28, 32)        128       
_________________________________________________________________
dropout_36 (Dropout)         (None, 28, 28, 32)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 28, 28, 64)        51264     
_________________________________________________________________
batch_normalization_73 (Batc (None, 28, 28, 64)        256       
_________________________________________________________________
conv3 (Conv2D)               (None, 28, 28, 32)        512

2021-09-17 13:26:18.931411: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-09-17 13:26:18.931428: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-09-17 13:26:18.933147: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


  1/782 [..............................] - ETA: 21:18 - loss: 2.9565 - categorical_accuracy: 0.2891 - sensitivity_at_specificity_5: 0.0000e+00

2021-09-17 13:26:20.688321: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-09-17 13:26:20.688336: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


  2/782 [..............................] - ETA: 8:43 - loss: 2.2149 - categorical_accuracy: 0.4180 - sensitivity_at_specificity_5: 0.0000e+00 

2021-09-17 13:26:21.449568: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-09-17 13:26:21.451103: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-09-17 13:26:21.454123: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./logs/toy_model2/20210917-132618/train/plugins/profile/2021_09_17_13_26_21

2021-09-17 13:26:21.455291: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to ./logs/toy_model2/20210917-132618/train/plugins/profile/2021_09_17_13_26_21/lucindazhaosmbp.lan.trace.json.gz
2021-09-17 13:26:21.459863: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./logs/toy_model2/20210917-132618/train/plugins/profile/2021_09_17_13_26_21

2021-09-17 13:26:21.460045: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to ./logs/toy_model2/

782/782 [==============================] - 464s 592ms/step - loss: 0.1319 - categorical_accuracy: 0.9537 - sensitivity_at_specificity_5: 0.9015 - val_loss: 0.1601 - val_categorical_accuracy: 0.9378 - val_sensitivity_at_specificity_5: 0.9551
Epoch 2/40
782/782 [==============================] - 447s 571ms/step - loss: 0.0752 - categorical_accuracy: 0.9735 - sensitivity_at_specificity_5: 0.9769 - val_loss: 0.4347 - val_categorical_accuracy: 0.8625 - val_sensitivity_at_specificity_5: 0.9671
Epoch 3/40
782/782 [==============================] - 425s 544ms/step - loss: 0.0504 - categorical_accuracy: 0.9820 - sensitivity_at_specificity_5: 0.9928 - val_loss: 0.0705 - val_categorical_accuracy: 0.9760 - val_sensitivity_at_specificity_5: 0.9459
Epoch 4/40
782/782 [==============================] - 425s 544ms/step - loss: 0.0432 - categorical_accuracy: 0.9847 - sensitivity_at_specificity_5: 0.9948 - val_loss: 0.3575 - val_categorical_accuracy: 0.8889 - val_sensitivity_at_specificity_5: 0.9464
Epo

KeyboardInterrupt: 

# Load Model and Run Predictions

In [ ]:
loaded_model = load_model('')


In [ ]:
loaded_model.predict(x_val[0:10,:,:,:])

In [ ]:
y_val[0:10, :]

# CNN + Xgboost

In [ ]:
for layer in my_model.layers:
    print (layer._name)

In [ ]:
new_output = my_model.get_layer('dense_2').output
new_output.shape

In [ ]:
new_model = Model(inputs=my_model.input, outputs=new_output)

In [ ]:
new_model.predict(x_val[0:10,:,:,:])

In [ ]:
# Use the output of new_model as input to Xgboost
# Equaivalently we are replacing the softmax regression in the original CNN with an Xgboost



In [ ]:
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import XGBClassifier


In [ ]:
xgb_x_train = new_model.predict(x_train)
xgb_x_val = new_model.predict(x_val)

In [ ]:
print(xgb_x_train.shape)
print(xgb_x_val.shape)

In [ ]:
xgb_model = XGBClassifier(
    base_score=0.5,
    objective='multi:softmax',
    num_class=6,
    
    gamma=1,
    max_depth=8,
    learning_rate=0.3, 
    min_child_weight=1,
    n_estimators=2000,
    subsample=0.9,
    colsample_bynode=1,
    colsample_bytree=0.9,
    reg_alpha=0,
    reg_lambda=1,

    random_state=0,
    booster='gbtree',
    tree_method='hist',
    seed=1234,
    missing=None,
    verbosity=1
)

print(xgb_model.get_xgb_params())



In [ ]:
trained = my_model.fit(
    tg[0], tg[1], early_stopping_rounds=10,
    # the last metric is used for early stopping
    eval_metric=["merror", "mlogloss"],
    eval_set=[tg, vg], verbose=True)

In [ ]:
import pickle
pickle.dump(trained, open('xgb_model', "wb"))